In [32]:
import pandas as pd
import xml.etree.ElementTree as ET

In [33]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"

pc = pd.read_csv('parking_space_count.csv')

In [34]:
with open(add_xml, "w") as add:
    print("""<?xml version="1.0" encoding="UTF-8"?>

<additional xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/additional_file.xsd">

    <location netOffset="-362563.54,-4832311.19" convBoundary="0.00,0.00,2064.54,1284.45" origBoundary="13.296064,43.631354,13.321826,43.642858" projParameter="+proj=utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"/>
    """, file=add)

In [35]:
def side_2_roadway(block_id, side, edge_oneway_dict):
    if not edge_oneway_dict[block_id]:
        if side == 'SE':
            return block_id + '-' + 'NE'
        elif side == 'NE':
            return block_id + '-' + 'NW'
        elif side == 'NW':
            return block_id + '-' + 'SW'
        elif side == 'SW':
            return block_id + '-' + 'SE'
    else: 
        return edge_oneway_dict[block_id]

In [36]:
edges = []

root = ET.parse(net_xml).getroot()
for child in root.iter('edge'):
    if child.get('function') != 'internal':
        edges.append(child.get('id'))
        
edge_oneway_dict = {}
for edge_id in edges:
    if edge_id not in edge_oneway_dict:
        # if not checked, go ahead to check oneway
        if sum(word.startswith(edge_id[:5]) for word in edges) == 2:
            edge_oneway_dict[edge_id[:5]] = 0
        else:
            assert sum(word.startswith(edge_id[:5]) for word in edges) == 1
            edge_oneway_dict[edge_id[:5]] = edge_id

In [37]:
for index, row in pc.iterrows():
    block_id = row['block_id']
    side = row['side']
    space_type = row['type']
    length = row['length']
    count = row['cnt']
    lane = side_2_roadway(block_id, side, edge_oneway_dict) + '_0'
    parking_area_id = str(block_id) + '-' + str(side)
    with open(add_xml, "a") as add:
        if space_type == 'PS':
            print('    <parkingArea id="{}" lane="{}" startPos="10" endPos="50" roadsideCapacity="{}" angle="0" />'.\
                  format(parking_area_id, lane, int(length)//16), # unit of curb length?
                  file=add)

In [38]:
with open(add_xml, "a") as add:
    print('\n</additional>', file = add)